In [ ]:
#!/usr/bin/env python312
# -*- coding: utf-8 -*-
"""
Algorithmic structure based on the K-DBSCAN: Kernel-Density-Based Spatial Clustering of Applications with Noise algorithm
[@author: Eduardo Pla-Sacristan (Created Tue Mar 6 14:54:00 2018)]

Theory from the following paper:
"K-DBSCAN: Identifying Spatial Clusters With Differing Density Levels"
[@authors: Madhuri Debnath, Praveen Kumar Tripathi, Ramez Elmasri]

Written by Elizabeth Ogunsanya for analysis carried out in Advanced Spatial Analytics at Columbia GSAPP. 
This methodology was not used in the final report, Organized Displacement: 
A Policing Legacy (A study of the sustained impacts of Oakland’s gang injunctions on Black youth). 
The analysis was conducted in collaboration with Ki-Sang Yi and Tim Yoshimura Small, current MS in Urban Planning candidates
at Columbia University Graduate School of Architecture, Planning and Preservation.

Data held in the following Github Repository: 
https://github.com/Tompotmelon/fantastic-adventure


"""

# ==================================================== IMPORTS ================================================ #

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib import cm
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from shapely import wkt
from shapely.geometry import Point
import os
from sklearn.impute import SimpleImputer
from scipy.spatial import cKDTree
from geopy.distance import geodesic

In [ ]:
# ============================================= LOADING IN DATA I =============================================== #

def load_data(path, lat_col='lat', lon_col='lon', crs='epsg:4326', geom_col=None):
    data = pd.read_csv(path)
    if geom_col:
        data[geom_col] = data[geom_col].apply(wkt.loads)
        gdf = gpd.GeoDataFrame(data, geometry=geom_col)
    else:
        gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data[lon_col], data[lat_col]))
    gdf.set_crs(crs, inplace=True)
    return gdf

In [ ]:
# ============================================= LOADING IN DATA II =============================================== #

base_path = 'Data/'

# CITY-WIDE
force2020_gdf = load_data(base_path + 'use_of_force_cleaned_2020_latlon.csv')
force2015_gdf = load_data(base_path + 'use_of_force_cleaned_2015_latlon.csv')
stops2015_gdf = load_data(base_path + 'stops_cleaned_2015_latlon.csv')
stops2020_gdf = load_data(base_path + 'stops_cleaned_2020_latlon.csv')
arrests2015_gdf = load_data(base_path + 'oakland_arrests_2015.csv', geom_col='Location')
arrests2020_gdf = load_data(base_path + 'oakland_arrests_2020.csv', geom_col='Location')
police_activity_2015 = gpd.read_file(base_path + 'police_activity_2015.shp')
police_activity_2020 = gpd.read_file(base_path + 'police_activity_2020.shp')
study_pop_block_2015 = gpd.read_file(base_path + 'StudyPop_perBlock_2015.shp')
study_pop_block_2020 = gpd.read_file(base_path + 'StudyPop_perBlock_2020.shp')

# WITHIN GIZs
force2020_northoak_gdf = gpd.read_file(base_path + 'oakland_use_of_force_2020_northoak_GIZ.shp')
force2015_northoak_gdf = gpd.read_file(base_path + 'oakland_use_of_force_2015_northoak_GIZ.shp')
force2020_fruitvale_gdf = gpd.read_file(base_path + 'oakland_use_of_force_2020_fruitvale_GIZ.shp')
force2015_fruitvale_gdf = gpd.read_file(base_path + 'oakland_use_of_force_2015_fruitvale_GIZ.shp')
stops2020_northoak_gdf = gpd.read_file(base_path + 'oakland_stops_2020_northoak_GIZ.shp')
stops2015_northoak_gdf = gpd.read_file(base_path + 'oakland_stops_2015_northoak_GIZ.shp')
stops2020_fruitvale_gdf = gpd.read_file(base_path + 'oakland_stops_2020_fruitvale_GIZ.shp')
stops2015_fruitvale_gdf = gpd.read_file(base_path + 'oakland_stops_2015_fruitvale_GIZ.shp')
arrests2020_northoak_gdf = gpd.read_file(base_path + 'oakland_arrests_2020_northoak_GIZ.shp')
arrests2015_northoak_gdf = gpd.read_file(base_path + 'oakland_arrests_2015_northoak_GIZ.shp')
arrests2020_fruitvale_gdf = gpd.read_file(base_path + 'oakland_arrests_2020_fruitvale_GIZ.shp')
arrests2015_fruitvale_gdf = gpd.read_file(base_path + 'oakland_arrests_2015_fruitvale_GIZ.shp')


In [ ]:
# Data and titles mapping
gdfs = {
    'force2020_gdf': force2020_gdf,
    'force2015_gdf': force2015_gdf,
    'stops2020_gdf': stops2020_gdf,
    'stops2015_gdf': stops2015_gdf,
    'arrests2015_gdf': arrests2015_gdf,
    'arrests2020_gdf': arrests2020_gdf,
    'police_activity_2015': police_activity_2015,
    'police_activity_2020': police_activity_2020,
    'force2020_northoak_gdf': force2020_northoak_gdf,
    'force2015_northoak_gdf': force2015_northoak_gdf,
    'force2020_fruitvale_gdf': force2020_fruitvale_gdf,
    'force2015_fruitvale_gdf': force2015_fruitvale_gdf,
    'stops2020_northoak_gdf': stops2020_northoak_gdf,
    'stops2015_northoak_gdf': stops2015_northoak_gdf,
    'stops2020_fruitvale_gdf': stops2020_fruitvale_gdf,
    'stops2015_fruitvale_gdf': stops2015_fruitvale_gdf,
    'arrests2020_northoak_gdf': arrests2020_northoak_gdf,
    'arrests2015_northoak_gdf': arrests2015_northoak_gdf,
    'arrests2020_fruitvale_gdf': arrests2020_fruitvale_gdf,
    'arrests2015_fruitvale_gdf': arrests2015_fruitvale_gdf,
    'study_pop_block_2015': study_pop_block_2015,
    'study_pop_block_2020': study_pop_block_2020
}

gdf_csv = {
    'force2020_gdf': force2020_gdf,
    'force2015_gdf': force2015_gdf,
    'stops2020_gdf': stops2020_gdf,
    'stops2015_gdf': stops2015_gdf,
    'arrests2015_gdf': arrests2015_gdf,
    'arrests2020_gdf': arrests2020_gdf,
}

# k = 4
gdf_csv_1 = {
    'force2020_gdf': force2020_gdf,
    'force2015_gdf': force2015_gdf,
    'stops2020_gdf': stops2020_gdf,
}


# k = 2
gdf_csv_2 = {
    'stops2015_gdf': stops2015_gdf,
    'arrests2015_gdf': arrests2015_gdf,
    'arrests2020_gdf': arrests2020_gdf,
}

gdf_shp = {
    'police_activity_2015': police_activity_2015,
    'police_activity_2020': police_activity_2020,
    'force2020_northoak_gdf': force2020_northoak_gdf,
    'force2015_northoak_gdf': force2015_northoak_gdf,
    'force2020_fruitvale_gdf': force2020_fruitvale_gdf,
    'force2015_fruitvale_gdf': force2015_fruitvale_gdf,
    'stops2020_northoak_gdf': stops2020_northoak_gdf,
    'stops2015_northoak_gdf': stops2015_northoak_gdf,
    'stops2020_fruitvale_gdf': stops2020_fruitvale_gdf,
    'stops2015_fruitvale_gdf': stops2015_fruitvale_gdf,
    'arrests2020_northoak_gdf': arrests2020_northoak_gdf,
    'arrests2015_northoak_gdf': arrests2015_northoak_gdf,
    'arrests2020_fruitvale_gdf': arrests2020_fruitvale_gdf,
    'arrests2015_fruitvale_gdf': arrests2015_fruitvale_gdf,
    # 'study_pop_block_2015': study_pop_block_2015,
    # 'study_pop_block_2020': study_pop_block_2020
}

titles = {
    'force2020_gdf': 'Use of Force 2020',
    'force2015_gdf': 'Use of Force 2015',
    'stops2020_gdf': 'Stops 2020',
    'stops2015_gdf': 'Stops 2015',
    'arrests2015_gdf': 'Arrests 2015',
    'arrests2020_gdf': 'Arrests 2020',
    'police_activity_2015': 'Aggregated Police Activity 2015',
    'police_activity_2020': 'Aggregated Police Activity 2020',
    'force2020_gdf': 'Use of Force 2020',
    'force2015_gdf': 'Use of Force 2015',
    'stops2020_gdf': 'Stops 2020',
    'stops2015_gdf': 'Stops 2015',
    'arrests2015_gdf': 'Arrests 2015',
    'arrests2020_gdf': 'Arrests 2020',
    'police_activity_2015': 'Aggregated Police Activity 2015',
    'police_activity_2020': 'Aggregated Police Activity 2020',
    'force2020_northoak_gdf': 'Use of Force in North Oakland 2020',
    'force2015_northoak_gdf': 'Use of Force in North Oakland 2015',
    'force2020_fruitvale_gdf': 'Use of Force in Fruitvale 2020',
    'force2015_fruitvale_gdf': 'Use of Force in Fruitvale 2015',
    'stops2020_northoak_gdf': 'Stops in North Oakland 2020',
    'stops2015_northoak_gdf': 'Stops in North Oakland 2015',
    'stops2015_fruitvale_gdf': 'Stops in Fruitvale 2015',
    'stops2020_fruitvale_gdf': 'Stops in Fruitvale 2020',
    'arrests2015_northoak_gdf': 'Arrests in North Oakland 2015',
    'arrests2020_northoak_gdf': 'Arrests in North Oakland 2020',
    'arrests2015_fruitvale_gdf': 'Arrests in Fruitvale 2015',
    'arrests2020_fruitvale_gdf': 'Arrests in Fruitvale 2020',
    'study_pop_block_2015': 'Study Population 2015',
    'study_pop_block_2020': 'Study Population 2020'
}


In [ ]:
#  =========================================== SET CRS TO EPSG:2227 ================================= #

for gdf_name, gdf in gdfs.items():
    if gdf.crs is not None and gdf.crs.to_string() != 'epsg:2227':
        gdf = gdf.to_crs('epsg:2227')  # Convert CRS if needed

    # Handle non-point geometries by using centroids
    if any(gdf.geom_type != 'Point'):
        gdf['geometry'] = gdf.geometry.centroid

    coords = np.array(list(zip(gdf.geometry.x, gdf.geometry.y)))

In [ ]:
plot_save_dir = 'C:/Users/elleo/OneDrive/Documents/Columbia GSAPP/Spring 2024/Advanced Spatial Analysis/Plots/Trial_4/'
os.makedirs(plot_save_dir, exist_ok=True)  # Creates the directory if it doesn't exist

In [ ]:
# ======================================== PRE-PROCESSING FOR K-MEANS ========================================= #

# Calculate l-densities in order to select appropriate k for k-means clustering
def calculate_ldensities(data, num_neighbors):
    """
    Calculate l-densities for given coordinate data, ensuring no division by zero and handling NaNs.
    """
    # Ensure data does not contain NaNs by using an imputer or dropping NaNs
    if np.isnan(data).any():
        imputer = SimpleImputer(strategy='mean')
        data = imputer.fit_transform(data)

    neigh = NearestNeighbors(n_neighbors=num_neighbors + 1)
    neigh.fit(data)
    distances, _ = neigh.kneighbors(data)
    distances = distances[:, 1:]  # Exclude the point itself
    with np.errstate(divide='ignore', invalid='ignore'):
        l_densities = 1 / np.mean(distances, where=(distances != 0), axis=1)
        l_densities[np.isinf(l_densities)] = 0  # Handle infinite values
    return l_densities

def plot_ldensities(l_densities, title, plot_save_dir):
    """
    Plot the sorted l-density values and save the plot.
    """
    sorted_l_densities = np.sort(l_densities)
    plt.scatter(range(len(sorted_l_densities)), sorted_l_densities, s=5)
    plt.xlabel('Points (sorted)')
    plt.ylabel('l-density')
    plt.title(f'Sorted l-densities: {title}')
    plt.savefig(f'{plot_save_dir}{title.replace(" ", "_")}_LDensities.png')
    plt.show()

def perform_kmeans(data, num_clusters):
    """
    Perform K-means clustering on the given data and return the cluster labels.
    """
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(data)
    return kmeans.labels_

def plot_kmeans(kmeans, title, plot_save_dir):
    """
    Plot k-means clusters and save the plot
    """
    plt.scatter(coords[:, 0], coords[:, 1], c= cluster_labels, cmap='viridis', s=8)
    plt.xlabel('Easting (m)')
    plt.ylabel('Northing (m)')
    plt.title(f'K-Means Clusters: {title}')
    plt.savefig(f'{plot_save_dir}{title.replace(" ", "_")}_kmeans.png')
    plt.savefig(f'{plot_save_dir}{title.replace(" ", "_")}_kmeans.svg')
    plt.show()

def export_to_geojson(self, data, labels, filename):
        """
        Export clustering results to a GeoJSON file.

        Parameters:
        data (numpy.ndarray): The dataset used for clustering.
        labels (numpy.ndarray): Cluster labels for each point.
        filename (str): File path where the GeoJSON will be saved.
        """
        gdf = gpd.GeoDataFrame(columns=['geometry', 'cluster'])
        gdf['geometry'] = [Point(xy) for xy in zip(data[:, 1], data[:, 0])]
        gdf['cluster'] = labels
        gdf.to_file(filename, driver='GeoJSON')


In [ ]:
#  =================================== L-DENSITIES CSV FILES =============================== #

# Calculate l-densities
for gdf_name, gdf in gdf_csv.items():
    if gdf.crs is not None and gdf.crs.to_string() != 'epsg:2227':
        gdf = gdf.to_crs('epsg:2227')  # Convert CRS if needed

    # Handle non-point geometries by using centroids
    if any(gdf.geom_type != 'Point'):
        gdf['geometry'] = gdf.geometry.centroid

    coords = np.array(list(zip(gdf.geometry.x, gdf.geometry.y)))

    # Calculate l-densities
    l_densities = calculate_ldensities(coords, 5)
    
    # Plot l-densities
    plot_ldensities(l_densities, gdf_name, plot_save_dir)

In [ ]:
#  =================================== L-DENSITIES SHP FILES =============================== #

# Calculate l-densities
for gdf_name, gdf in gdf_shp.items():
    if gdf.crs is not None and gdf.crs.to_string() != 'epsg:2227':
        gdf = gdf.to_crs('epsg:2227')  # Convert CRS if needed

    # Handle non-point geometries by using centroids
    if any(gdf.geom_type != 'Point'):
        gdf['geometry'] = gdf.geometry.centroid

    coords = np.array(list(zip(gdf.geometry.x, gdf.geometry.y)))

    # Calculate l-densities
    l_densities = calculate_ldensities(coords, 5)
    
    # Plot l-densities
    plot_ldensities(l_densities, gdf_name, plot_save_dir)

In [ ]:
#  ================================== K-MEANS CSV FILES =================================== #
for gdf_name, gdf in gdf_csv_1.items():
    title = titles[gdf_name]
    coords = np.array(list(zip(gdf.geometry.x, gdf.geometry.y)))

    # Ensure there are no NaNs in your coordinates
    coords = np.nan_to_num(coords, nan=np.nanmean(coords))  # Replace NaNs with the mean of columns

    num_clusters = 4  # Adjust based on your needs
    cluster_labels = perform_kmeans(coords, num_clusters)
    plot_kmeans(cluster_labels, title, plot_save_dir)

for gdf_name, gdf in gdf_csv_2.items():
    title = titles[gdf_name]
    coords = np.array(list(zip(gdf.geometry.x, gdf.geometry.y)))

    # Ensure there are no NaNs in your coordinates
    coords = np.nan_to_num(coords, nan=np.nanmean(coords))  # Replace NaNs with the mean of columns

    num_clusters = 2  # Adjust based on your needs
    cluster_labels = perform_kmeans(coords, num_clusters)
    plot_kmeans(cluster_labels, title, plot_save_dir)

In [ ]:

gdf_3 = {
    'police_activity_2015': police_activity_2015,
    'police_activity_2020': police_activity_2020,
    'force2020_northoak_gdf': force2020_northoak_gdf,
    'arrests2020_northoak_gdf': arrests2020_northoak_gdf,
    'arrests2020_gdf': arrests2020_gdf,
    'stops2015_gdf': stops2015_gdf,
    # 'study_pop_block_2015': study_pop_block_2015,
    # 'study_pop_block_2020': study_pop_block_2020
}

# l > 15
gdf_3a = {
        'arrests2015_northoak_gdf': arrests2015_northoak_gdf,
        'arrests2015_fruitvale_gdf': arrests2015_fruitvale_gdf,

}

gdf_4 = {
    'force2015_fruitvale_gdf': force2015_fruitvale_gdf,
    'stops2020_northoak_gdf': stops2020_northoak_gdf,
}

gdf_5 = {
    'stops2015_fruitvale_gdf': stops2015_fruitvale_gdf,
    'arrests2015_fruitvale_gdf': arrests2015_fruitvale_gdf,
    'stops2020_fruitvale_gdf': stops2020_fruitvale_gdf,
    'force2020_gdf': force2020_gdf,
    'stops2020_gdf': stops2020_gdf,


}

gdf_6 = {
    'force2015_northoak_gdf': force2015_northoak_gdf,
    'arrests2015_northoak_gdf': arrests2015_northoak_gdf,
    'arrests2020_fruitvale_gdf': arrests2020_fruitvale_gdf,
    'arrests2015_gdf': arrests2015_gdf,
    'stops2015_northoak_gdf': stops2015_northoak_gdf,
}

gdf_7 = {
    'force2020_fruitvale_gdf': force2020_fruitvale_gdf,
    'force2015_gdf': force2015_gdf,
}


In [ ]:
# ======================================== K-MEANS SHP FILES ================================== #
for gdf_name, gdf in gdf_3.items():
    title = titles[gdf_name]
    coords = np.array(list(zip(gdf.geometry.x, gdf.geometry.y)))

    # Ensure there are no NaNs in your coordinates
    coords = np.nan_to_num(coords, nan=np.nanmean(coords))  # Replace NaNs with the mean of columns

    num_clusters =   # Adjust based on your needs
    cluster_labels = perform_kmeans(coords, num_clusters)
    plot_kmeans(cluster_labels, title, plot_save_dir)


In [ ]:
for gdf_name, gdf in gdf_3.items():
    # Convert all geometries to points (e.g., using centroid if they are polygons)
    gdf['geometry'] = gdf.geometry.centroid
    
    title = titles[gdf_name]
    coords = np.array(list(zip(gdf.geometry.x, gdf.geometry.y)))

    # Ensure there are no NaNs in your coordinates
    coords = np.nan_to_num(coords, nan=np.nanmean(coords))  # Replace NaNs with the mean of columns

    num_clusters = 3  # Adjust based on your needs
    cluster_labels = perform_kmeans(coords, num_clusters)
    plot_kmeans(cluster_labels, title, plot_save_dir)

for gdf_name, gdf in gdf_4.items():
    # Convert all geometries to points (e.g., using centroid if they are polygons)
    gdf['geometry'] = gdf.geometry.centroid

    title = titles[gdf_name]
    coords = np.array(list(zip(gdf.geometry.x, gdf.geometry.y)))

    # Ensure there are no NaNs in your coordinates
    coords = np.nan_to_num(coords, nan=np.nanmean(coords))  # Replace NaNs with the mean of columns

    num_clusters = 4  # Adjust based on your needs
    cluster_labels = perform_kmeans(coords, num_clusters)
    plot_kmeans(cluster_labels, title, plot_save_dir)

for gdf_name, gdf in gdf_5.items():
    # Convert all geometries to points (e.g., using centroid if they are polygons)
    gdf['geometry'] = gdf.geometry.centroid

    title = titles[gdf_name]
    coords = np.array(list(zip(gdf.geometry.x, gdf.geometry.y)))

    # Ensure there are no NaNs in your coordinates
    coords = np.nan_to_num(coords, nan=np.nanmean(coords))  # Replace NaNs with the mean of columns

    num_clusters = 5  # Adjust based on your needs
    cluster_labels = perform_kmeans(coords, num_clusters)
    plot_kmeans(cluster_labels, title, plot_save_dir)

for gdf_name, gdf in gdf_6.items():
    # Convert all geometries to points (e.g., using centroid if they are polygons)
    gdf['geometry'] = gdf.geometry.centroid

    title = titles[gdf_name]
    coords = np.array(list(zip(gdf.geometry.x, gdf.geometry.y)))

    # Ensure there are no NaNs in your coordinates
    coords = np.nan_to_num(coords, nan=np.nanmean(coords))  # Replace NaNs with the mean of columns

    num_clusters = 6  # Adjust based on your needs
    cluster_labels = perform_kmeans(coords, num_clusters)
    plot_kmeans(cluster_labels, title, plot_save_dir)

for gdf_name, gdf in gdf_7.items():
    # Convert all geometries to points (e.g., using centroid if they are polygons)
    gdf['geometry'] = gdf.geometry.centroid
    
    coords = np.array(list(zip(gdf.geometry.x, gdf.geometry.y)))
    title = titles[gdf_name]
    coords = np.array(list(zip(gdf.geometry.x, gdf.geometry.y)))

    # Ensure there are no NaNs in your coordinates
    coords = np.nan_to_num(coords, nan=np.nanmean(coords))  # Replace NaNs with the mean of columns

    num_clusters = 7  # Adjust based on your needs
    cluster_labels = perform_kmeans(coords, num_clusters)
    plot_kmeans(cluster_labels, title, plot_save_dir)

In [ ]:

class KDBSCAN:
    def __init__(self, radius_func, num_neighbors=20):
        self.radius_func = radius_func
        self.num_neighbors = num_neighbors

    def radius_func(self, coords, point_index, l=20):
        tree = cKDTree(coords)
        distances, indexes = tree.query(coords[point_index], k=l+1)
        geodesic_distances = [geodesic((coords[point_index][1], coords[point_index][0]),
                                        (coords[idx][1], coords[idx][0])).meters for idx in indexes[1:]]
        geodesic_distances.sort()
        epsilon_i = geodesic_distances[-1]
        return epsilon_i

    def fit(self, gdf, num_clusters):
        if gdf.crs.to_string() != 'epsg:2227':
            gdf = gdf.to_crs('epsg:2227')
        gdf['geometry'] = gdf.geometry.centroid
        coords = np.array(list(zip(gdf.geometry.x, gdf.geometry.y)))
        kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(coords)
        self.density_levels = kmeans.labels_

        gdf_4326 = gdf.to_crs('epsg:4326')
        coords_4326 = np.array(list(zip(gdf_4326.geometry.x, gdf_4326.geometry.y)))
        self.CL = np.zeros(len(coords), dtype=int)
        visited = set()
        cluster_id = 1

        for i, point in enumerate(coords):
            if i not in visited:
                visited.add(i)
                εi = self.radius_func(coords_4326, i, l=5)
                NeighborPts = self.regionQuery(coords, i, εi)
                if len(NeighborPts) > self.num_neighbors:
                    self.expandCluster(coords, coords_4326, i, NeighborPts, cluster_id, visited)
                    cluster_id += 1

        gdf['cluster'] = self.CL
        return gdf

    def regionQuery(self, coords, idx, εi):
        distances = np.linalg.norm(coords - coords[idx], axis=1)
        return np.where((distances <= εi) & (self.density_levels == self.density_levels[idx]))[0]

    def expandCluster(self, coords, coords_4326, idx, NeighborPts, cluster_id, visited):
        self.CL[idx] = cluster_id
        points_to_evaluate = list(NeighborPts)
        i = 0

        while i < len(points_to_evaluate):
            point = points_to_evaluate[i]
            if point not in visited:
                visited.add(point)
                self.CL[point] = cluster_id
                εi = self.radius_func(coords_4326, point, l=5)
                new_points = self.regionQuery(coords, point, εi)

                if len(new_points) >= self.num_neighbors:
                    points_to_evaluate.extend([np for np in new_points if np not in visited and np not in points_to_evaluate])

            i += 1

    def calculate_ldensities(self, data, num_neighbors):
        neigh = NearestNeighbors(n_neighbors=num_neighbors + 1)
        neigh.fit(data)
        distances, _ = neigh.kneighbors(data)
        distances = distances[:, 1:]  # Exclude the point itself
        l_densities = np.zeros(data.shape[0])
        for i, dist in enumerate(distances):
            if np.any(dist > 0):
                l_densities[i] = 1 / np.mean(dist[dist > 0])
            else:
                l_densities[i] = 0
        return l_densities

    def save_geojson(self, gdf, filename):
        gdf.to_file(filename, driver='GeoJSON')


RUN K-DBSCAN

In [ ]:
# k = 3
for gdf_name, gdf in gdf_3.items():
    title = titles[gdf_name]

    num_clusters = 3  # k
    kdb = KDBSCAN(radius_func, num_neighbors=5)
    clustered_gdf = kdb.fit(gdf)
    # clustered_gdf.to_crs('epsg:4326', inplace=True)  # Convert back to WGS 84 for GeoJSON compatibility
    kdb.save_geojson(clustered_gdf, f'C:/Users/elleo/OneDrive/Documents/Columbia GSAPP/Spring 2024/Advanced Spatial Analysis/Plots/Trial_4/{title.replace(" ", "_")}_clustered_KDBSCAN_data.geojson')

    # Plot with dynamic title
    plot_kdbscan(clustered_gdf, gdf_name, titles)

In [ ]:
# k = 4
for gdf_name, gdf in gdf_4.items():
    title = titles[gdf_name]

    num_clusters = 4  # k
    kdb = KDBSCAN(radius_func, num_neighbors=5)
    # Call fit with the GeoDataFrame and num_clusters
    clustered_gdf = kdb.fit(gdf, num_clusters)
    # clustered_gdf.to_crs('epsg:4326', inplace=True)  # Convert back to WGS 84 for GeoJSON compatibility
    kdb.save_geojson(clustered_gdf, f'C:/Users/elleo/OneDrive/Documents/Columbia GSAPP/Spring 2024/Advanced Spatial Analysis/Plots/Trial_4/{title.replace(" ", "_")}_clustered_KDBSCAN_data.geojson')

    # Plot with dynamic title
    plot_kdbscan(clustered_gdf, gdf_name, titles)

In [ ]:
# k = 5
for gdf_name, gdf in gdf_5.items():
    title = titles[gdf_name]

    num_clusters = 5  # k
    kdb = KDBSCAN(radius_func, num_neighbors=5)
    clustered_gdf = kdb.fit(gdf)
    # clustered_gdf.to_crs('epsg:4326', inplace=True)  # Convert back to WGS 84 for GeoJSON compatibility
    kdb.save_geojson(clustered_gdf, f'C:/Users/elleo/OneDrive/Documents/Columbia GSAPP/Spring 2024/Advanced Spatial Analysis/Plots/Trial_4/{title.replace(" ", "_")}_clustered_KDBSCAN_data.geojson')

    # Plot with dynamic title
    plot_kdbscan(clustered_gdf, gdf_name, titles)